# 最长上升子序列

**题目来源：力扣（LeetCode）**

**链接：https://leetcode-cn.com/problems/longest-increasing-subsequence/**

## 一、题目

给你一个整数数组 nums ，找到其中最长严格递增子序列的长度。

子序列 是由数组派生而来的序列，删除（或不删除）数组中的元素而不改变其余元素的顺序。例如，[3,6,2,7] 是数组 [0,3,1,6,2,2,7] 的子序列。

### 提示：

- 1 <= nums.length <= 2500
- -10^4 <= nums[i] <= 10^4

## 二、示例

### 示例 1：

输入：nums = [10,9,2,5,3,7,101,18]

输出：4

解释：最长递增子序列是 [2,3,7,101]，因此长度为 4 。

### 示例 2：

输入：nums = [0,1,0,3,2,3]

输出：4

### 示例 3：

输入：nums = [7,7,7,7,7,7,7]

输出：1

### 进阶: 你能将算法的时间复杂度降低到 O(n log n) 吗?

## 三、解题思路

### 一、暴力枚举

In [1]:
def lengthOfLIS(nums):
    def generate(item, i, result):
        if i >= len(nums):
            if not result or len(result[-1]) < len(item):
                result.append(item[:])
            return
        num = nums[i]
        if not item or num > item[-1]:
            item.append(num)
            generate(item, i + 1, result)
            item.pop()
        generate(item, i + 1, result)
    result = []
    generate([], 0, result)
    return len(result[-1]) if result else 0

In [2]:
lengthOfLIS([10,9,2,5,3,7,101,18])

4

### 二、动态规划

- 若dp[i]代表当前i个数字中最长上升子序列的长度，可否找出dp[i]和dp[i-1]之间的关系？
- 若dp[i]代表以第i个数字为结尾的最长上升子序列的长度，可否找出dp[i]和dp[i-1]之间的关系？

第一种很难推出递推关系，那么我们采用第二种方式进行递推，假设除了最后一位为结尾的dp[i]都已求得

过程模拟：
```
dp[0] = 1 ([10])
dp[1] = 1 ([9])
dp[2] = 1 ([2])
dp[3] = 2 ([2, 5])
dp[4] = 2 ([2, 3])
dp[5] = 3 ([2, 5, 7]/[2, 3, 7])
dp[6] = 4 ([2, 5, 7, 101]/[2, 3, 7, 101])
```
那么dp[7]怎么求呢？

找到对应的nums[j] (j < i)使得 nums[i] > nums[j]， 取最大的那个+1

即：

```
大于dp[0]对应的nums[0]，则：[10] + [18]
大于dp[1]对应的nums[1]，则：[9] + [18]
大于dp[2]对应的nums[2]，则：[2] + [18]
大于dp[3]对应的nums[3]，则：[2, 5] + [18]
大于dp[4]对应的nums[4]，则：[2, 3] + [18]
大于dp[5]对应的nums[5]，则：[2, 5, 7] + [18]/[2, 3, 7]+[18]
小于dp[6]对应的nums[6]，则：[18]

则取最大的+1，即dp[7] = dp[5] + 1
```

最后找出dp数组中的最大值返回即可

In [1]:
def lengthOfLIS(nums):
    if not nums:
        return 0
    dp = [1] * (len(nums) + 1)
    maxLen = 1
    for i in range(1, len(nums)):
        for j in range(i):
            if nums[i] > nums[j]:
                dp[i] = max(dp[i], dp[j] + 1)
        maxLen = max(dp[i], maxLen)
    return maxLen

In [2]:
lengthOfLIS([10,9,2,5,3,7,101,18])

4

### 三、利用栈

设置一个栈 stack，stack[i]代表长度为 i+1 的上升子序列最后一个元素的**最小可能取值**，即若要组成长度为 i+2 的上升子序列，需要一个大于stack[i]的元素。最终栈的大小即为最长上升子序列的长度。

[1, 3, 2, 3, 1, 4]

长度为1的上升子序列：[1],[2],[3],[4]

长度为2的上升子序列：[1, 2], [1, 3], [1, 4], [2, 3], [2, 4], [3, 4]

长度为3的上升子序列：[1, 2, 3],[1, 2, 4],[2, 3, 4]

长度为4的上升子序列：[1, 2, 3, 4]

1. 设置一个栈，stack，将nums[0]push进栈

2. 从1到n-1遍历数组nums：若nums[i]大于栈顶，则将其push进栈，否则，从栈底遍历到栈顶，若遍历时，栈中元素大于等于nums[i]，使用其替换该元素，跳出循环

3. 返回栈的大小

In [1]:
def lengthOfLIS(nums):
    if not nums:
        return 0
    stack = [nums[0]]
    for i in range(1, len(nums)):
        if nums[i] > stack[-1]:
            stack.append(nums[i])
        else:
            for j in range(len(stack)):
                if stack[j] >= nums[i]:
                    stack[j] = nums[i]
                    break
    return len(stack)

In [2]:
lengthOfLIS([10,9,2,5,3,7,101,18])

4